#

In [1]:
import os
from PIL import Image

# Функции

In [2]:
def background2any(fname, new=(0, 0, 0, 0), old=None, delete=False, new_fname=None):
    
    img = Image.open(f'{fname}').convert("RGBA")
    if delete:
        os.remove(fname)

    size = img.size
    pics = list(img.getdata())
    
    if old is None:
        old = pics[0]
    
    new_img = [new] * size[0] * size[1]
    for y in range(size[1]):
        left = y * size[0]
        right = (y + 1) * size[0] - 1
        while left < right and pics[left] == old:
            left += 1
        while left < right and pics[right] == old:
            right -= 1
        for i in range(left, right):
            new_img[i] = pics[i]

    img.putdata(new_img)

    if new_fname is None:
        new_fname = '.'.join(fname.split('.')[:-1]) + '_b2a.png'

    img.save(new_fname, 'PNG')

In [3]:
def delete_border(fname, wid=2, back=None, delete=False, new_fname=None):
    
    img = Image.open(f'{fname}').convert("RGBA")
    if delete:
        os.remove(fname)
  
    size = img.size
    pics = list(img.getdata())
    
    if back is None:
        back = pics[0]
    
    new_img = [back] * size[0] * size[1]

    for y in range(size[1]):
        left = y * size[0]
        right = (y + 1) * size[0] - 1
        while left+wid < right-wid and pics[left] == back:
            left += 1
        while left+wid < right-wid and pics[right] == back:
            right -= 1
        for i in range(left+wid, right-wid):
            new_img[i] = pics[i]
        
    for x in range(size[0]):
        up = x
        down = size[0] * (size[1]-1) + x
        while up + wid*size[0] < down-wid*size[0] and pics[up] == back:
            up += size[0]
        while up + wid*size[0] < down-wid*size[0] and pics[down] == back:
            down -= size[0]
        for i in range(wid):
            new_img[up + i*size[0]] = back
            new_img[down - i*size[0]] = back

    img.putdata(new_img)

    if new_fname is None:
        new_fname = '.'.join(fname.split('.')[:-1]) + '_db.png'

    img.save(new_fname, 'PNG')

In [4]:
def crop_box(fname, wid=8, back=None, delete=False, new_fname=None):
    
    img = Image.open(f'{fname}').convert("RGBA")
    if delete:
        os.remove(fname)
    
    size = img.size
    pics = list(img.getdata())

    if back is None:
        back = pics[0]    
    
    min_left = size[0]
    min_right = size[0]
    for y in range(size[1]):
        left = 0
        right = 0
        while y*size[0] + left < (y+1)*size[0]-1 - right and pics[y*size[0] + left] == back:
            left += 1
        while y*size[0] + left < (y+1)*size[0]-1 - right and pics[(y+1)*size[0]-1 - right] == back:
            right += 1
        if left and right:
            min_left = min(min_left, left)
            min_right = min(min_right, right)

    img = img.crop((min_left-wid, 0, size[0]+wid-min_right, size[1]))
    
    if new_fname is None:
        new_fname = '.'.join(fname.split('.')[:-1]) + '_cb.png'

    img.save(new_fname, 'PNG')

In [5]:
def full_change(fname, delete=False, new_fname=None):
    
    background2any(fname, delete=delete)
    
    fname = '.'.join(fname.split('.')[:-1])
    
    delete_border(f'{fname}_b2a.png', delete=True)
    
    if new_fname is None:
        new_fname = fname + '_fc.png'

    crop_box(f'{fname}_b2a_db.png', delete=True, new_fname=new_fname)

# Тест функций

In [6]:
background2any('example.png')

In [7]:
delete_border('example.png')

In [8]:
crop_box('example.png')

In [9]:
full_change('example.png')

In [10]:
background2any('example.png')
delete_border('example_b2a.png', delete=True)
crop_box('example_b2a_db.png', delete=True)

In [11]:
background2any('example2.jpg')

In [12]:
delete_border('example2.jpg')

In [13]:
crop_box('example2.jpg')

In [14]:
background2any('example2.jpg')
delete_border('example2_b2a.png', delete=True)
crop_box('example2_b2a_db.png', delete=True)